In [ ]:
# Project 1: Password Strength Analyzer + Claude Explainer

Authored and Created by Robert Gravelle

**Overview**  
This notebook evaluates password strength using the `zxcvbn` library (realistic entropy and pattern detection) and uses Claude (Anthropic LLM) to provide friendly, educational explanations and improvement suggestions.

**Key Features**
- Technical metrics: strength score (0-4), estimated crack time, warnings, suggestions
- Interactive UI for easy testing
- Positive, encouraging feedback generated by Claude
- Simple visual representation of password strength

**Technologies**  
Python, zxcvbn, Anthropic Claude API, ipywidgets, matplotlib

**Important**  
Do **NOT** enter real passwords — use only for demonstration purposes!

Date: January 2026

In [3]:
import os
from pathlib import Path
from dotenv import load_dotenv

from anthropic import Anthropic
from zxcvbn import zxcvbn

import ipywidgets as widgets
from IPython.display import display, clear_output, Markdown

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

print("All required packages imported successfully!")

All required packages imported successfully!


In [4]:
# Load environment variables (expects .env file in project root)
load_dotenv()

# Initialize Anthropic client (automatically uses ANTHROPIC_API_KEY from env)
client = Anthropic()

print("Anthropic client initialized ✓")

Anthropic client initialized ✓


In [5]:
def ask_claude(prompt, model="claude-sonnet-4-5", max_tokens=700):
    """
    Send prompt to Claude and return the response text.
    Uses the latest Sonnet model as of January 2026.
    """
    try:
        response = client.messages.create(
            model=model,
            max_tokens=max_tokens,
            temperature=0.45,
            messages=[{"role": "user", "content": prompt}]
        )
        return response.content[0].text.strip()
    except Exception as e:
        return f"Error calling Claude: {str(e)}\n(Check API key, credits, model name, or Anthropic status)"

In [6]:
def check_password(password):
    """
    Evaluate password strength using zxcvbn.
    Returns dictionary with key metrics.
    """
    if not password.strip():
        raise ValueError("Password cannot be empty")

    result = zxcvbn(password)

    return {
        'score': result['score'],                    # 0-4
        'crack_time': result['crack_times_display']['offline_fast_hashing_1e10_per_second'],
        'warning': result.get('feedback', {}).get('warning', 'No specific warnings'),
        'suggestions': result.get('feedback', {}).get('suggestions', []),
        'full_result': result
    }

In [7]:
def plot_strength_bar(score):
    """Display a simple horizontal bar showing password strength (0-4)"""
    colors = ['#d32f2f', '#f57c00', '#fbc02d', '#7cb342', '#388e3c']  # red to green
    fig, ax = plt.subplots(figsize=(7, 0.8))
    
    ax.barh([0], [score+0.3], color=colors[score], height=0.6)
    ax.set_xlim(0, 4.5)
    ax.set_xticks(range(5))
    ax.set_yticks([])
    ax.set_xlabel('Strength Score (0-4)')
    ax.set_title('Password Strength', fontsize=11, pad=8)
    
    # Add score number in the middle of the bar
    ax.text(score/2 + 0.1, 0, str(score), va='center', ha='center', 
            color='white' if score <= 2 else 'black', fontweight='bold')
    
    plt.tight_layout()
    plt.show()

In [12]:
def on_analyze_clicked(b):
    with output:
        clear_output(wait=True)
       
        pwd = password_input.value.strip()
        if not pwd:
            display(Markdown("**Please enter a password**"))
            return
       
        try:
            analysis = check_password(pwd)
            score = analysis['score']
           
            # Display results
            display(Markdown(f"**Strength Score:** {score}/4"))
            plot_strength_bar(score)
           
            display(Markdown(f"**Estimated crack time (fast attack):** {analysis['crack_time']}"))
           
            if analysis['warning'] and analysis['warning'] != 'No specific warnings':
                display(Markdown(f"**Warning:** {analysis['warning']}"))
           
            if analysis['suggestions']:
                display(Markdown("**Suggestions:**"))
                for suggestion in analysis['suggestions']:
                    print(f"• {suggestion}")
           
            print("\n" + "─" * 70 + "\n")
           
            # Prepare prompt for Claude (Option 2 - neutral, factual report)
            prompt = f"""You are a security assessment system.

Password analysis results:
- Strength score: {score}/4
- Offline crack time (fast hashing): {analysis['crack_time']}
- Warning message: {analysis['warning']}
- Suggested improvements: {', '.join(analysis['suggestions']) if analysis['suggestions'] else 'None'}

Provide a direct, factual report:
1. Current security status
2. Primary weaknesses
3. Recommended password patterns (2–3 examples, prefer passphrases)
Do not include motivational language or personal opinions."""
           
            display(Markdown("**Claude's analysis report:**"))
            response = ask_claude(prompt)
            display(Markdown(response))
           
        except Exception as e:
            display(Markdown(f"**Error:** {str(e)}"))

analyze_button.on_click(on_analyze_clicked)

# Layout
ui = widgets.VBox([
    widgets.Label("Enter a test password:", style=dict(font_weight='bold', font_size='14px')),
    password_input,
    analyze_button,
    output
], layout=widgets.Layout(padding='15px', width='500px'))

display(ui)

## Portfolio Note

Created by Robert Gravelle as part of the 2026 Cybersecurity Portfolio. 

Contact: coiweb.developer.1@gmail.com
    
GitHub: https://github.com/Rob-Gravelle